## 환율 데이터 처리하기

In [16]:
import pandas as pd
import numpy as np
# csv 파일 가져오기
exchange_df = pd.read_csv('../public/kor_exchange.csv')

# 데이터가 반전된 것을 확인함
exchange_df.head()

,Unnamed: 0,Unnamed: 1,1970,1971,1972,1973,1974,1975,1976,1977,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,원/달러,환율,316.7,373.3,398.9,397.5,484.0,484.0,484.0,484.0,...,"1,099.3","1,172.5","1,207.7","1,070.5","1,115.7","1,156.4","1,086.3","1,188.8","1,264.5","1,288.0"
1,NaN,절상율(%),-3.9,-15.2,-6.4,0.4,-17.9,0.0,0.0,0.0,...,-4.0,-6.2,-2.9,12.8,8.2,-3.5,6.5,-8.6,-6.0,-1.8
2,엔/달러,환율,357.7,314.8,302.0,280.0,301.0,305.2,292.8,240.0,...,120.4,120.4,116.6,112.8,110.6,109.1,103.3,115.1,133.7,141.2
3,NaN,절상율(%),0.0,13.6,4.2,7.9,-7.0,-1.4,4.2,22.0,...,-12.5,0.0,3.2,3.4,2.0,1.3,5.6,-10.2,-13.9,-5.3
4,원/100엔,환율,88.5,118.6,132.1,142.0,160.8,158.6,165.3,201.7,...,913.0,974.1,"1,035.4",949.3,"1,008.9","1,059.8","1,051.2","1,032.5",945.6,912.3


In [17]:
# 첫 번째 열을 'Date'로 이름 변경
exchange_df.columns = ['dummy', '속성'] + [str(year) for year in range(1970, 2024)]

# 속성이라는 인덱스 지정 후 데이터 행 열 반전
df_transposed = exchange_df.set_index('속성').transpose()

# 인덱스 명 변경 -> 속성 to 연도
df_transposed.rename_axis('연도', inplace=True)

# 컬럼 이름 재지정
df_transposed.columns = ['미국환율', '미국 절산율', '일본환율', '일본 절산율', '중국환율', '중국 절산율']

# inplace: 기존 데이터에 영향 주기
df_transposed.drop('dummy', axis=0, inplace=True)

# 컬럼 가져와서 모두 다 숫자형으로 변경
for key in df_transposed.columns:
    # regex: 정규표현식
    # replace 는 문자열 치환
    # astype 타입 변경
    # 환율의 object 자료형을 float 으로 변경 
    df_transposed[key] = df_transposed[key].replace({',': ''}, regex=True).astype(float)

# 10개의 대표 데이터 가져오기
df_transposed.head(10)

,미국환율,미국 절산율,일본환율,일본 절산율,중국환율,중국 절산율
연도,,,,,,
1970,316.7,-3.9,357.7,0.0,88.5,-3.9
1971,373.3,-15.2,314.8,13.6,118.6,-25.3
1972,398.9,-6.4,302.0,4.2,132.1,-10.2
1973,397.5,0.4,280.0,7.9,142.0,-7.0
1974,484.0,-17.9,301.0,-7.0,160.8,-11.7
1975,484.0,0.0,305.2,-1.4,158.6,1.4
1976,484.0,0.0,292.8,4.2,165.3,-4.0
1977,484.0,0.0,240.0,22.0,201.7,-18.0
1978,484.0,0.0,194.6,23.3,248.7,-18.9


In [18]:
# 결측치 확인 -> 빈 데이터 없음
df_transposed.isnull().sum()

# 데이터 타입 확인 모두 float 타입
print(df_transposed.dtypes)

미국환율      float64
미국 절산율    float64
일본환율      float64
일본 절산율    float64
중국환율      float64
중국 절산율    float64
dtype: object


## 소비자 물가 지수 처리하기

In [19]:
# csv 파일 가져오기
cpi = pd.read_csv('../public/cpi.csv')

# 행 열을 돌려 반전시킨다
cpi = cpi.transpose()

for i in range(2, len(cpi.columns)):
    del cpi[i]

# 행렬 반전시 이상해지는 데이터 지우기
cpi = cpi.drop('단위:', axis=0)
cpi = cpi.drop('2020=100, %', axis=0)

# 컬럼 이름 지정하기
cpi.columns = ['소비자물가지수', '소비자물가상승률']

# 컬럼 인덱스 이름 변경하기
# 처리한 환율 DataFrame 과 같은 인덱스 이름을 지정해줌
cpi.rename_axis("연도", inplace=True)

# 컬럼을 가져와서 숫자형으로 변환하는 반복문
for key in cpi.columns:
    # 쉼표를 지우기 
    # 데이터 타입을 float(숫자형)으로 변경하기
    cpi[key] = cpi[key].astype(float)

# 대푯값 10개 가져오기
cpi

,소비자물가지수,소비자물가상승률
연도,,
1970,5.3,13.5
1971,5.9,11.7
1972,6.1,3.2
1973,7.6,24.3
1974,9.5,25.2
1975,10.9,15.3
1976,12.0,10.1
1977,13.8,14.5
1978,16.3,18.3


In [20]:
# 데이터의 결측치 확인
cpi.isnull().sum()

소비자물가지수     1
소비자물가상승률    1
dtype: int64

# 결측치 결과
결측치를 확인하니, 소비자 물가지수, 소비자 물가 상승률에 1개씩 결측값이 존재하는걸 확인함

# 해결방안
소비자 물가지수, 상승률에만 결측값이 존재한다.
 -> 특정 날짜에 데이터 측정 결과가 포함되지 않았다는것을 인지함.

따라서, dropna axis=0 을 통해 열이 아닌 행만 지우기로 함. (해당 열 물가 지수, 상승률은 꼭 필요한 데이터이기 때문)

In [21]:
cpi.dropna(axis=0, inplace=True)

cpi

,소비자물가지수,소비자물가상승률
연도,,
1970,5.3,13.5
1971,5.9,11.7
1972,6.1,3.2
1973,7.6,24.3
1974,9.5,25.2
1975,10.9,15.3
1976,12.0,10.1
1977,13.8,14.5
1978,16.3,18.3


In [22]:
# 두개의 데이터 프레임을 열 기준으로 병합
result_frame = pd.concat([cpi, df_transposed], axis=1)

result_frame.isnull().sum()

소비자물가지수     1
소비자물가상승률    1
미국환율        0
미국 절산율      0
일본환율        0
일본 절산율      0
중국환율        0
중국 절산율      0
dtype: int64

In [23]:
# None 이 포함된 행 지우기
result_frame.dropna(axis=0, inplace=True)

# 잘 지워졌는지 결측치 확인
result_frame.isnull().sum()

소비자물가지수     0
소비자물가상승률    0
미국환율        0
미국 절산율      0
일본환율        0
일본 절산율      0
중국환율        0
중국 절산율      0
dtype: int64

In [24]:
result_frame

,소비자물가지수,소비자물가상승률,미국환율,미국 절산율,일본환율,일본 절산율,중국환율,중국 절산율
연도,,,,,,,,
1970,5.3,13.5,316.7,-3.9,357.7,0.0,88.5,-3.9
1971,5.9,11.7,373.3,-15.2,314.8,13.6,118.6,-25.3
1972,6.1,3.2,398.9,-6.4,302.0,4.2,132.1,-10.2
1973,7.6,24.3,397.5,0.4,280.0,7.9,142.0,-7.0
1974,9.5,25.2,484.0,-17.9,301.0,-7.0,160.8,-11.7
1975,10.9,15.3,484.0,0.0,305.2,-1.4,158.6,1.4
1976,12.0,10.1,484.0,0.0,292.8,4.2,165.3,-4.0
1977,13.8,14.5,484.0,0.0,240.0,22.0,201.7,-18.0
1978,16.3,18.3,484.0,0.0,194.6,23.3,248.7,-18.9
